In [1]:
import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
WAVENET_SEARCH_SPACE = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : scope.int(hp.quniform('num_blocks', 2, 20, 2)),
        'block_filters' : scope.int(hp.quniform('block_filters', 16, 256, 16)),
        'residual_filters' : scope.int(hp.quniform('residual_filters', 8, 128, 8)),
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    }
}


fit_args = {
    'batch_size' : 16,
    'epochs' : 20,
    'class_weighted' : True
}
CHKPT_FNAME = 'temp_best_weights.h5'


def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    p = {**WAVENET_SEARCH_SPACE, **params}
    p['network_args']['num_blocks'] = int(params['network_args']['num_blocks'])
    p['network_args']['block_filters'] = int(params['network_args']['block_filters'])
    p['network_args']['residual_filters'] = int(params['network_args']['residual_filters'])
    return p

In [3]:
def run_experiment(num_train, max_evals=15, search_space=WAVENET_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=["pseudoGR"],
                              pseudoGR_args={'features' : ['mean', 'var'], 
                                             'per_channel' : True}) for s in train_sets[:2]]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for dset in datasets:
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=WAVENET_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [4]:
trials9, best_params9 = run_experiment(9)

Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('pseudoGR', (11210, 32, 8))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('pseudoGR', (9807, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,))

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


...

                                                                                 
33/65 [==============>...............]                                           
 - ETA: 2s - loss: 0.6892 - acc: 0.6707                                          
                                                                                 
34/65 [==============>...............]                                           
 - ETA: 2s - loss: 0.6891 - acc: 0.6713                                          
                                                                                 
35/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6874 - acc: 0.6705                                          
                                                                                 
36/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6856 - acc: 0.6729                                          
                

 - 12s 181ms/step - loss: 0.6889 - acc: 0.6699 - val_loss: 0.7106 - val_acc: 0.6377

Epoch 7/20                                                                       
 1/65 [..............................]                                           
 - ETA: 5s - loss: 0.8664 - acc: 0.6401                                          
                                                                                 
 2/65 [..............................]                                           
 - ETA: 5s - loss: 0.7417 - acc: 0.6782                                          
                                                                                 
 3/65 [>.............................]                                           
 - ETA: 4s - loss: 0.7057 - acc: 0.6751                                          
                                                                                 
 4/65 [>.............................]                                           
 - ETA: 4s - 

 - ETA: 2s - loss: 0.6922 - acc: 0.6760                                          
                                                                                 
34/65 [==============>...............]                                           
 - ETA: 2s - loss: 0.6907 - acc: 0.6764                                          
                                                                                 
35/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6975 - acc: 0.6740                                          
                                                                                 
36/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6914 - acc: 0.6764                                          
                                                                                 
37/65 [================>.............]                                           
 - ETA: 2s - los

 1/65 [..............................]                                           
 - ETA: 4s - loss: 0.5387 - acc: 0.7451                                          
                                                                                 
 2/65 [..............................]                                           
 - ETA: 4s - loss: 0.4959 - acc: 0.7720                                          
                                                                                 
 3/65 [>.............................]                                           
 - ETA: 4s - loss: 0.6613 - acc: 0.6965                                          
                                                                                 
 4/65 [>.............................]                                           
 - ETA: 4s - loss: 0.6589 - acc: 0.6860                                          
                                                                                 
 5/65 [=>.......

34/65 [==============>...............]                                           
 - ETA: 2s - loss: 0.6540 - acc: 0.6814                                          
                                                                                 
35/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6538 - acc: 0.6832                                          
                                                                                 
36/65 [===============>..............]                                           
 - ETA: 2s - loss: 0.6514 - acc: 0.6841                                          
                                                                                 
37/65 [================>.............]                                           
 - ETA: 2s - loss: 0.6586 - acc: 0.6807                                          
                                                                                 
38/65 [=========

In [5]:
trials9.results

[{'loss': 1.1289731045555418,
  'acc_scores': [0.6400037031893718, 0.691],
  'f1_scores': [0.4596651827404754, 0.5132270418316919],
  'log_losses': [1.2111431370276236, 1.0468030720834602],
  'status': 'ok'},
 {'loss': 1.1764952439610357,
  'acc_scores': [0.6361153543489331, 0.74425],
  'f1_scores': [0.499351648221784, 0.5030423128419639],
  'log_losses': [1.2553113219240044, 1.097679165998067],
  'status': 'ok'},
 {'loss': 1.1729461999507054,
  'acc_scores': [0.6087811877979911, 0.6863333333333334],
  'f1_scores': [0.4340401864106761, 0.48806387076645485],
  'log_losses': [1.247329445823488, 1.0985629540779227],
  'status': 'ok'},
 {'loss': 1.1518313981108599,
  'acc_scores': [0.6294033236124612, 0.7130416666666667],
  'f1_scores': [0.4865809948986636, 0.5422880168503095],
  'log_losses': [1.2221433612117232, 1.0815194350099966],
  'status': 'ok'},
 {'loss': 1.1858433898744245,
  'acc_scores': [0.5917928065546452, 0.7114583333333333],
  'f1_scores': [0.4871858042029258, 0.468024825490

In [6]:
best_params9

{'block_filters': 80.0, 'num_blocks': 18.0, 'residual_filters': 16.0}

In [7]:
trials9.best_trial

{'state': 2,
 'tid': 6,
 'spec': None,
 'result': {'loss': 1.0628825079628759,
  'acc_scores': [0.6478498356709717, 0.7158333333333333],
  'f1_scores': [0.4864561229043998, 0.5603366687217408],
  'log_losses': [1.1600832164436325, 0.9656817994821194],
  'status': 'ok'},
 'misc': {'tid': 6,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'block_filters': [6], 'num_blocks': [6], 'residual_filters': [6]},
  'vals': {'block_filters': [80.0],
   'num_blocks': [18.0],
   'residual_filters': [16.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2019, 7, 15, 20, 33, 53, 284000),
 'refresh_time': datetime.datetime(2019, 7, 15, 20, 41, 49, 597000)}

In [4]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        
        model.fit(dset, callbacks=[chkpt_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [6]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 18,
        'block_filters' : 80,
        'residual_filters' : 16,
        'output_resolution' : 32,
        'dropout_rate' : 0.5
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

preds_df_9 = pred_results_df(9, params)
preds_df_9.to_csv('pgr_wavenet_results_train_sets9.csv')

Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('pseudoGR', (11210, 32, 8))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('pseudoGR', (9807, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,))

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]


...

Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1884,)), ('top', (1884,)), ('base', (1884,)), ('pseudoGR', (1884, 32, 8))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('pseudoGR', (11210, 32, 8))]
Shapes of `(batch_X, batch_y)`: (16, 4096, 8), (16, 128, 5)
Epoch 1/20
64/65 [============================>.] - ETA: 0s - loss: 1.2094 - acc: 0.3940
Epoch 00001: val_loss improved from inf to 1.15518, saving model to temp_best_weights.h5
65/6

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base'

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('pseudoGR', (9807, 32, 8))]
Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]
Shapes of `(batch_X, batch_y)`: (16, 4096, 8), (16, 128, 5)
Epoch 1/20
65/66 [============================>.] - ETA: 0s - loss: 1.2381 - acc: 0.4631
Epoch 00001: val_loss improved from inf to 1.07024, saving model to temp_best_weights.h5
66/66 [===

In [7]:
accuracy_score(preds_df_9.y_pred.values, preds_df_9.y_true.values)

0.6079373951928452